# DistilBERT Fine-tuning on IMDB Dataset

**Assignment 3: Fine-tuning Pretrained Transformers**

This notebook demonstrates two fine-tuning strategies:
1. **Full fine-tuning**: Update all model parameters
2. **LoRA**: Parameter-efficient fine-tuning

---

## ⚙️ Setup

**Before running:**
- Make sure you're using a GPU runtime
- Go to: `Runtime` → `Change runtime type` → Select `T4 GPU`

In [3]:
# Install required packages
!pip install -q transformers datasets peft accelerate evaluate scikit-learn

In [4]:
# Import libraries
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import time
import json

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB


## 📝 Configuration

Adjust these parameters as needed:

In [5]:
class Config:
    # Model settings
    model_name = "distilbert-base-uncased"
    num_labels = 2
    max_length = 512

    # Training settings
    batch_size = 16          # Reduce to 8 if OOM
    num_epochs = 3
    learning_rate = 2e-5
    weight_decay = 0.01

    # LoRA settings
    lora_r = 8
    lora_alpha = 16
    lora_dropout = 0.1

    # Data settings (use smaller for quick testing)
    train_size = 5000        # Full: 25000
    test_size = 1000         # Full: 25000

    # Output
    output_dir_full = "./results_full_finetuning"
    output_dir_lora = "./results_lora_finetuning"
    random_seed = 42

config = Config()

# Set seeds
torch.manual_seed(config.random_seed)
np.random.seed(config.random_seed)

print("✓ Configuration set")

✓ Configuration set


## 📊 Load and Prepare Data

In [6]:
# Load IMDB dataset
print("Loading IMDB dataset...")
dataset = load_dataset("imdb")

# Use subset for faster training
if config.train_size < len(dataset['train']):
    dataset['train'] = dataset['train'].shuffle(seed=config.random_seed).select(range(config.train_size))
if config.test_size < len(dataset['test']):
    dataset['test'] = dataset['test'].shuffle(seed=config.random_seed).select(range(config.test_size))

print(f"Train samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"\nSample review: {dataset['train'][0]['text'][:200]}...")

Loading IMDB dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Train samples: 5000
Test samples: 1000

Sample review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. F...


In [7]:
# Tokenize
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=config.max_length
    )

print("Tokenizing...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("✓ Data prepared")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizing...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

✓ Data prepared


## 🎯 Define Metrics

In [8]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

print("✓ Metrics defined")

✓ Metrics defined


---

## 🔵 Method 1: Full Fine-tuning

Updates all 66M parameters

In [9]:
print("="*70)
print("FULL FINE-TUNING")
print("="*70)

# Load model
model_full = AutoModelForSequenceClassification.from_pretrained(
    config.model_name,
    num_labels=config.num_labels
)

# Count parameters
total_params = sum(p.numel() for p in model_full.parameters())
trainable_params = sum(p.numel() for p in model_full.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

FULL FINE-TUNING


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total parameters: 66,955,010
Trainable parameters: 66,955,010
Trainable %: 100.00%


In [10]:
# Training arguments
training_args_full = TrainingArguments(
    output_dir=config.output_dir_full,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=100,
    seed=config.random_seed,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# Create trainer
trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer initialized")

/tmp/ipython-input-1971539941.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(


✓ Trainer initialized


In [11]:
# Train (this will take ~15-20 minutes)
print("\nStarting training...")
start_time = time.time()
train_result_full = trainer_full.train()
full_training_time = time.time() - start_time

print(f"\n✓ Training completed in {full_training_time:.2f}s ({full_training_time/60:.2f} min)")


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.293000,0.365891,0.852000,0.865942
2,0.173800,0.309243,0.893000,0.894581
3,0.118100,0.349523,0.900000,0.899598



✓ Training completed in 256.66s (4.28 min)


In [12]:
# Evaluate
eval_result_full = trainer_full.evaluate()

print("\n" + "="*70)
print("FULL FINE-TUNING RESULTS")
print("="*70)
print(f"Accuracy: {eval_result_full['eval_accuracy']:.4f}")
print(f"F1 Score: {eval_result_full['eval_f1']:.4f}")
print(f"Training Time: {full_training_time:.2f}s")
print("="*70)


FULL FINE-TUNING RESULTS
Accuracy: 0.9000
F1 Score: 0.8996
Training Time: 256.66s


In [13]:
# ==========================================
# ==========================================
print("\n" + "="*70)
print("ERROR ANALYSIS - FULL FINE-TUNING")
print("="*70)

predictions_full = trainer_full.predict(tokenized_datasets["test"])
y_pred_full = np.argmax(predictions_full.predictions, axis=-1)

y_true = np.array(tokenized_datasets["test"]["label"])

error_indices = np.where(y_pred_full != y_true)[0]
print(f"\nTotal errors: {len(error_indices)} out of {len(y_true)} ({len(error_indices)/len(y_true)*100:.2f}%)")

false_positives = np.where((y_pred_full == 1) & (y_true == 0))[0]
false_negatives = np.where((y_pred_full == 0) & (y_true == 1))[0]

print(f"\nError breakdown:")
print(f"  False Positives (predicted positive, actually negative): {len(false_positives)}")
print(f"  False Negatives (predicted negative, actually positive): {len(false_negatives)}")

print("\n" + "-"*70)
print("SAMPLE ERROR CASES:")
print("-"*70)

print("\n🔴 False Positives (Model predicted POSITIVE, but actually NEGATIVE):")
print("-"*70)
for i, idx in enumerate(false_positives[:5]):
    text = dataset['test'][int(idx)]['text']
    print(f"\n[{i+1}] Review (truncated):")
    print(f"{text[:300]}...")
    print(f"True label: NEGATIVE | Predicted: POSITIVE")

print("\n" + "-"*70)
print("\n🔵 False Negatives (Model predicted NEGATIVE, but actually POSITIVE):")
print("-"*70)
for i, idx in enumerate(false_negatives[:5]):
    text = dataset['test'][int(idx)]['text']
    print(f"\n[{i+1}] Review (truncated):")
    print(f"{text[:300]}...")
    print(f"True label: POSITIVE | Predicted: NEGATIVE")

probabilities_full = torch.softmax(torch.tensor(predictions_full.predictions), dim=-1).numpy()

confident_errors = []
for idx in error_indices:
    confidence = np.max(probabilities_full[idx])
    if confidence > 0.8:
        confident_errors.append({
            'idx': int(idx),
            'confidence': float(confidence),
            'predicted': int(y_pred_full[idx]),
            'true': int(y_true[idx])
        })

print("\n" + "="*70)
print(f"\nHigh-confidence errors (>80% confidence): {len(confident_errors)}")
if len(confident_errors) > 0:
    print("\nTop 3 high-confidence errors:")
    confident_errors = sorted(confident_errors, key=lambda x: x['confidence'], reverse=True)
    for i, err in enumerate(confident_errors[:3]):
        text = dataset['test'][err['idx']]['text']
        print(f"\n[{i+1}] Confidence: {err['confidence']:.2%}")
        print(f"Review: {text[:200]}...")
        print(f"True: {'POSITIVE' if err['true']==1 else 'NEGATIVE'} | Predicted: {'POSITIVE' if err['predicted']==1 else 'NEGATIVE'}")

print("="*70)


ERROR ANALYSIS - FULL FINE-TUNING



Total errors: 100 out of 1000 (10.00%)

Error breakdown:
  False Positives (predicted positive, actually negative): 60
  False Negatives (predicted negative, actually positive): 40

----------------------------------------------------------------------
SAMPLE ERROR CASES:
----------------------------------------------------------------------

🔴 False Positives (Model predicted POSITIVE, but actually NEGATIVE):
----------------------------------------------------------------------

[1] Review (truncated):
These days, writers, directors and producers are relying more and more on the "surprise" ending. The old art of bringing a movie to closure, taking all of the information we have learned through out the movie and bringing it to a nice complete ending, has been lost. Now what we have is a movie that,...
True label: NEGATIVE | Predicted: POSITIVE

[2] Review (truncated):
A holiday on a boat, a married couple, an angry waiter and a shipwreck is the reason to this films beginning.<br /><b

---

## 🔴 Method 2: LoRA Fine-tuning

Updates only ~0.5% of parameters

In [14]:
print("="*70)
print("LoRA FINE-TUNING")
print("="*70)

# Load fresh model
model_lora = AutoModelForSequenceClassification.from_pretrained(
    config.model_name,
    num_labels=config.num_labels
)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    lora_dropout=config.lora_dropout,
    target_modules=["q_lin", "v_lin"],
    bias="none",
)

# Apply LoRA
model_lora = get_peft_model(model_lora, lora_config)
model_lora.print_trainable_parameters()

# Count parameters
trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)
param_reduction = (1 - trainable_params_lora / trainable_params) * 100

print(f"\nParameter reduction: {param_reduction:.2f}%")

LoRA FINE-TUNING


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925

Parameter reduction: 98.90%


In [16]:
# Training arguments
training_args_lora = TrainingArguments(
    output_dir=config.output_dir_lora,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=100,
    seed=config.random_seed,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# Create trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Trainer initialized")

✓ Trainer initialized


/tmp/ipython-input-321065280.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


In [17]:
# Train (this will take ~10-15 minutes)
print("\nStarting training...")
start_time = time.time()
train_result_lora = trainer_lora.train()
lora_training_time = time.time() - start_time

print(f"\n✓ Training completed in {lora_training_time:.2f}s ({lora_training_time/60:.2f} min)")


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.571200,0.460924,0.847000,0.849558
2,0.345600,0.331921,0.854000,0.852525
3,0.319600,0.324932,0.858000,0.854806



✓ Training completed in 179.42s (2.99 min)


In [18]:
# Evaluate
eval_result_lora = trainer_lora.evaluate()

print("\n" + "="*70)
print("LoRA FINE-TUNING RESULTS")
print("="*70)
print(f"Accuracy: {eval_result_lora['eval_accuracy']:.4f}")
print(f"F1 Score: {eval_result_lora['eval_f1']:.4f}")
print(f"Training Time: {lora_training_time:.2f}s")
print("="*70)


LoRA FINE-TUNING RESULTS
Accuracy: 0.8580
F1 Score: 0.8548
Training Time: 179.42s


In [19]:
print("\n" + "="*70)
print("ERROR ANALYSIS - LoRA")
print("="*70)

predictions_lora = trainer_lora.predict(tokenized_datasets["test"])
y_pred_lora = np.argmax(predictions_lora.predictions, axis=-1)

# y_true = np.array(tokenized_datasets["test"]["label"])

error_indices_lora = np.where(y_pred_lora != y_true)[0]
print(f"\nTotal errors: {len(error_indices_lora)} out of {len(y_true)} ({len(error_indices_lora)/len(y_true)*100:.2f}%)")


false_positives_lora = np.where((y_pred_lora == 1) & (y_true == 0))[0]
false_negatives_lora = np.where((y_pred_lora == 0) & (y_true == 1))[0]

print(f"\nError breakdown:")
print(f"  False Positives: {len(false_positives_lora)}")
print(f"  False Negatives: {len(false_negatives_lora)}")

print("\n" + "-"*70)
print("SAMPLE ERROR CASES:")
print("-"*70)

print("\n🔴 False Positives (Model predicted POSITIVE, but actually NEGATIVE):")
print("-"*70)
for i, idx in enumerate(false_positives_lora[:5]):
    text = dataset['test'][int(idx)]['text']
    print(f"\n[{i+1}] Review (truncated):")
    print(f"{text[:300]}...")
    print(f"True label: NEGATIVE | Predicted: POSITIVE")

print("\n" + "-"*70)
print("\n🔵 False Negatives (Model predicted NEGATIVE, but actually POSITIVE):")
print("-"*70)
for i, idx in enumerate(false_negatives_lora[:5]):
    text = dataset['test'][int(idx)]['text']
    print(f"\n[{i+1}] Review (truncated):")
    print(f"{text[:300]}...")
    print(f"True label: POSITIVE | Predicted: NEGATIVE")

probabilities_lora = torch.softmax(torch.tensor(predictions_lora.predictions), dim=-1).numpy()

confident_errors_lora = []
for idx in error_indices_lora:
    confidence = np.max(probabilities_lora[idx])
    if confidence > 0.8:
        confident_errors_lora.append({
            'idx': int(idx),
            'confidence': float(confidence),
            'predicted': int(y_pred_lora[idx]),
            'true': int(y_true[idx])
        })

print("\n" + "="*70)
print(f"\nHigh-confidence errors (>80% confidence): {len(confident_errors_lora)}")
if len(confident_errors_lora) > 0:
    print("\nTop 3 high-confidence errors:")
    confident_errors_lora = sorted(confident_errors_lora, key=lambda x: x['confidence'], reverse=True)
    for i, err in enumerate(confident_errors_lora[:3]):
        text = dataset['test'][err['idx']]['text']
        print(f"\n[{i+1}] Confidence: {err['confidence']:.2%}")
        print(f"Review: {text[:200]}...")
        print(f"True: {'POSITIVE' if err['true']==1 else 'NEGATIVE'} | Predicted: {'POSITIVE' if err['predicted']==1 else 'NEGATIVE'}")

print("="*70)


ERROR ANALYSIS - LoRA



Total errors: 142 out of 1000 (14.20%)

Error breakdown:
  False Positives: 72
  False Negatives: 70

----------------------------------------------------------------------
SAMPLE ERROR CASES:
----------------------------------------------------------------------

🔴 False Positives (Model predicted POSITIVE, but actually NEGATIVE):
----------------------------------------------------------------------

[1] Review (truncated):
These days, writers, directors and producers are relying more and more on the "surprise" ending. The old art of bringing a movie to closure, taking all of the information we have learned through out the movie and bringing it to a nice complete ending, has been lost. Now what we have is a movie that,...
True label: NEGATIVE | Predicted: POSITIVE

[2] Review (truncated):
A holiday on a boat, a married couple, an angry waiter and a shipwreck is the reason to this films beginning.<br /><br />I like boobs. No question about that. But when the main character allies wit

---

## 📊 Comparison

In [20]:
import pandas as pd

# Create comparison table
comparison_df = pd.DataFrame({
    'Method': ['Full Fine-tuning', 'LoRA'],
    'Trainable Params': [f"{trainable_params:,}", f"{trainable_params_lora:,}"],
    'Trainable %': [f"{100:.2f}%", f"{100 * trainable_params_lora / trainable_params:.2f}%"],
    'Training Time (s)': [f"{full_training_time:.2f}", f"{lora_training_time:.2f}"],
    'Accuracy': [f"{eval_result_full['eval_accuracy']:.4f}", f"{eval_result_lora['eval_accuracy']:.4f}"],
    'F1 Score': [f"{eval_result_full['eval_f1']:.4f}", f"{eval_result_lora['eval_f1']:.4f}"]
})

print("\n" + "="*100)
print("COMPARISON TABLE")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)

# Key insights
speedup = full_training_time / lora_training_time
accuracy_diff = eval_result_lora['eval_accuracy'] - eval_result_full['eval_accuracy']

print(f"\n📊 Key Insights:")
print(f"  • LoRA reduces trainable parameters by {param_reduction:.2f}%")
print(f"  • LoRA is {speedup:.2f}x faster")
print(f"  • Accuracy difference: {accuracy_diff:+.4f}")
print(f"  • LoRA achieves comparable performance with 99%+ fewer trainable parameters!")


COMPARISON TABLE
          Method Trainable Params Trainable % Training Time (s) Accuracy F1 Score
Full Fine-tuning       66,955,010     100.00%            256.66   0.9000   0.8996
            LoRA          739,586       1.10%            179.42   0.8580   0.8548

📊 Key Insights:
  • LoRA reduces trainable parameters by 98.90%
  • LoRA is 1.43x faster
  • Accuracy difference: -0.0420
  • LoRA achieves comparable performance with 99%+ fewer trainable parameters!


## 💾 Save Results

In [21]:
# Save comprehensive results
results_summary = {
    "dataset": "IMDB",
    "model": config.model_name,
    "train_samples": len(dataset['train']),
    "test_samples": len(dataset['test']),
    "full_finetuning": {
        "trainable_parameters": int(trainable_params),
        "training_time_seconds": float(full_training_time),
        "accuracy": float(eval_result_full['eval_accuracy']),
        "f1_score": float(eval_result_full['eval_f1']),
    },
    "lora_finetuning": {
        "trainable_parameters": int(trainable_params_lora),
        "training_time_seconds": float(lora_training_time),
        "accuracy": float(eval_result_lora['eval_accuracy']),
        "f1_score": float(eval_result_lora['eval_f1']),
        "lora_config": {
            "r": config.lora_r,
            "alpha": config.lora_alpha,
            "dropout": config.lora_dropout,
        }
    },
    "comparison": {
        "parameter_reduction_percent": float(param_reduction),
        "speedup": float(speedup),
        "accuracy_difference": float(accuracy_diff),
    }
}

with open("results_summary.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("✓ Results saved to results_summary.json")
print("\n🎉 Experiment completed successfully!")

✓ Results saved to results_summary.json

🎉 Experiment completed successfully!


## 📥 Download Results

Download the results file to your local machine:

In [22]:
from google.colab import files

# Download results
files.download('results_summary.json')
print("✓ Results downloaded")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Results downloaded
